In [50]:
import os, sys
sys.path.append(os.path.dirname(os.path.abspath(os.getcwd())))

from src.configuration.constants import PROCESSED_DATA_DIRECTORY, ROOT_DIRECTORY

import logging
import random
import numpy as np
import pickle
from numpy import expand_dims, zeros, ones, asarray
from numpy.random import randn, randint

from matplotlib import pyplot

import tensorflow as tf
from functools import partial

from tensorflow.keras.layers import *
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import plot_model
from tensorflow.keras import backend as K


logging.getLogger('tensorflow').setLevel(logging.ERROR)  # suppress warnings

https://machinelearningmastery.com/semi-supervised-generative-adversarial-network/

# Loading the dataset

In [6]:
with open(os.path.join(PROCESSED_DATA_DIRECTORY, 'MSL.pickle'), 'rb') as f:
    msl_dataset = pickle.load(f)
    
X_train = msl_dataset['y_train']
y_train = msl_dataset['y_train']
labels_train = np.array([random.choice([0, 1]) for i in range(2109)])

X_train.shape, y_train.shape, labels_train.shape

((2109, 100, 1), (2109, 100, 1), (2109,))

# Phase 1: Semi-supervised GAN

## Discriminator

### Separate Discriminator Models With Shared Weights


#### Encoder

In [30]:
def build_encoder(input_shape, lstm_units, dense_units, encoder_reshape_shape):
    x = Input(shape=input_shape)
    model = Sequential()
    model.add(Bidirectional(LSTM(units=lstm_units, return_sequences=True)))
    model.add(Flatten())
    model.add(Dense(units=dense_units))
    model.add(Reshape(target_shape=encoder_reshape_shape))
    return Model(x, model(x))

#### Generator

In [31]:
def build_generator(input_shape, generator_reshape_dim, generator_reshape_shape):
    x = Input(shape=input_shape)
    model = Sequential()
    model.add(Flatten())
    model.add(Dense(units=generator_reshape_dim))
    model.add(Reshape(target_shape=generator_reshape_shape))
    model.add(Bidirectional(LSTM(units=64, return_sequences=True, dropout=0.2, recurrent_dropout=0.2), merge_mode='concat'))
    model.add(UpSampling1D(size=2))
    model.add(Bidirectional(LSTM(units=64, return_sequences=True, dropout=0.2, recurrent_dropout=0.2), merge_mode='concat'))
    model.add(TimeDistributed(Dense(units=1)))
    model.add(Activation(activation='tanh'))
    return Model(x, model(x))
              

#### Critic X

In [52]:
def build_critic_x(input_shape, n_classes):
    x = Input(shape=input_shape)
    model = Sequential()
    
    for _ in range(4):
        model.add(Conv1D(filters=64, kernel_size=5))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(rate=0.25))
    
    model.add(Flatten())
    
    # Unsupervised model
    unsup_out = Dense(1, activation='sigmoid')(model(x))
    d_unsup_model = Model(x, unsup_out)
    
    # Supervised model
    sup_out = Dense(n_classes + 1, activation='softmax')(model(x))
    d_sup_model = Model(x, sup_out)
    
    return d_unsup_model, d_sup_model

#### Critic Z

In [53]:
def build_critic_z(input_shape, dense_units=20):
    x = Input(shape=input_shape)
    model = Sequential()
    model.add(Flatten())
    
    for _ in range(2):
        model.add(Dense(units=dense_units))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(rate=0.2))
        
    model.add(Dense(units=1))
    return Model(x, model(x))

#### Hyper-parameters

In [54]:
input_shape=X_train[0].shape
target_shape=y_train[0].shape
latent_dim=20
learning_rate=0.0005
epochs=70
batch_size=64
iterations_critic=5
latent_shape = (latent_dim, 1)
n_classes = 2

shape = np.asarray(X_train)[0].shape
length = shape[0]
target_shape = np.asarray(y_train)[0].shape


generator_reshape_dim = length // 2
generator_reshape_shape = (length // 2, 1)
encoder_reshape_shape = latent_shape

encoder_input_shape = shape
generator_input_shape = latent_shape
critic_x_input_shape = target_shape
critic_z_input_shape = latent_shape

lstm_units = 100
dense_units = 20

optimizer = tf.keras.optimizers.Adam(learning_rate)

In [55]:
encoder = build_encoder(
    input_shape=encoder_input_shape,
    lstm_units=lstm_units,
    dense_units=dense_units,
    encoder_reshape_shape=encoder_reshape_shape,
)

generator = build_generator(
    input_shape=generator_input_shape,
    generator_reshape_dim=generator_reshape_dim,
    generator_reshape_shape=generator_reshape_shape,
)

critic_x_unsup, critic_x_sup = build_critic_x(
    input_shape=critic_x_input_shape,
    n_classes=n_classes
)

critic_z = build_critic_z(
    input_shape=critic_z_input_shape,
)

In [56]:
def _wasserstein_loss(y_true, y_pred):
    return K.mean(y_true * y_pred)

def _gradient_penalty_loss(y_true, y_pred, averaged_samples):
    gradients = K.gradients(y_pred, averaged_samples)[0]
    gradients_sqr = K.square(gradients)
    gradients_sqr_sum = K.sum(gradients_sqr, axis=np.arange(1, len(gradients_sqr.shape)))
    gradient_l2_norm = K.sqrt(gradients_sqr_sum)
    gradient_penalty = K.square(1 - gradient_l2_norm)
    return K.mean(gradient_penalty)

In [57]:
generator.trainable = False
encoder.trainable = False

x = Input(shape=input_shape)
y = Input(shape=target_shape)
z = Input(shape=(latent_dim, 1))

x_ = generator(z)
z_ = encoder(x)
fake_x = critic_x_unsup(x_) # Fake
valid_x = critic_x_unsup(y) # Truth

label = critic_x_sup(y)

#### Critic X Unsupervised Model

In [59]:
alpha = K.random_uniform((64, 1, 1))
interpolated_x = (alpha * [y, x_][0]) + ((1 - alpha) * [y, x_][1])
validity_interpolated_x = critic_x_unsup(interpolated_x)
partial_gp_loss_x = partial(_gradient_penalty_loss, averaged_samples=interpolated_x)
partial_gp_loss_x.__name__ = 'gradient_penalty'
critic_x_unsup_model = Model(inputs=[y, z], outputs=[valid_x, fake_x,validity_interpolated_x])
critic_x_unsup_model.compile(loss=[_wasserstein_loss, _wasserstein_loss, partial_gp_loss_x], 
                       optimizer=optimizer, loss_weights=[1, 1, 10])
critic_x_unsup_model.summary()

Model: "model_33"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_45 (InputLayer)           [(None, 20, 1)]      0                                            
__________________________________________________________________________________________________
input_44 (InputLayer)           [(None, 100, 1)]     0                                            
__________________________________________________________________________________________________
model_28 (Model)                (None, 100, 1)       133787      input_45[0][0]                   
__________________________________________________________________________________________________
tf_op_layer_mul_6 (TensorFlowOp [(64, 100, 1)]       0           input_44[0][0]                   
___________________________________________________________________________________________

#### Critic X Supervised Model

In [63]:
critic_x_sup.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=optimizer, 
    metrics=['accuracy'],
)
critic_x_sup_model = critic_x_sup
critic_x_sup_model.summary()

Model: "model_30"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_41 (InputLayer)        [(None, 100, 1)]          0         
_________________________________________________________________
sequential_28 (Sequential)   multiple                  62016     
_________________________________________________________________
dense_48 (Dense)             (None, 3)                 16131     
Total params: 78,147
Trainable params: 78,147
Non-trainable params: 0
_________________________________________________________________


#### Critic Z Model

In [65]:
fake_z = critic_z(z_)
valid_z = critic_z(z)
alpha = K.random_uniform((64, 1, 1))
interpolated_z = (alpha * [z, z_][0]) + ((1 - alpha) * [z, z_][1])
validity_interpolated_z = critic_z(interpolated_z)
partial_gp_loss_z = partial(_gradient_penalty_loss, averaged_samples=interpolated_z)
partial_gp_loss_z.__name__ = 'gradient_penalty'
critic_z_model = tf.keras.Model(inputs=[x, z], outputs=[valid_z, fake_z,validity_interpolated_z])
critic_z_model.compile(loss=[_wasserstein_loss, _wasserstein_loss,
                                  partial_gp_loss_z], optimizer=optimizer,
                            loss_weights=[1, 1, 10])
critic_z_model.summary()

Model: "model_35"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_43 (InputLayer)           [(None, 100, 1)]     0                                            
__________________________________________________________________________________________________
input_45 (InputLayer)           [(None, 20, 1)]      0                                            
__________________________________________________________________________________________________
model_27 (Model)                (None, 20, 1)        481620      input_43[0][0]                   
__________________________________________________________________________________________________
tf_op_layer_mul_10 (TensorFlowO [(64, 20, 1)]        0           input_45[0][0]                   
___________________________________________________________________________________________

#### Encoder Generator Model

In [68]:
critic_x_sup.trainable = False
critic_x_unsup.trainable = False
critic_z.trainable = False
generator.trainable = True
encoder.trainable = True

z_gen = Input(shape=(latent_dim, 1))
x_gen_ = generator(z_gen)
x_gen = Input(shape=input_shape)
z_gen_ = encoder(x_gen)
x_gen_rec = generator(z_gen_)
fake_gen_x = critic_x_unsup(x_gen_)
fake_gen_z = critic_z(z_gen_)

encoder_generator_model = Model([x_gen, z_gen], [fake_gen_x, fake_gen_z, x_gen_rec])
encoder_generator_model.compile(loss=[_wasserstein_loss, _wasserstein_loss,'mse'], 
                                optimizer=optimizer,
                                loss_weights=[1, 1, 10])

encoder_generator_model.summary()

Model: "model_37"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_48 (InputLayer)           [(None, 20, 1)]      0                                            
__________________________________________________________________________________________________
input_49 (InputLayer)           [(None, 100, 1)]     0                                            
__________________________________________________________________________________________________
model_28 (Model)                (None, 100, 1)       133787      input_48[0][0]                   
                                                                 model_27[3][0]                   
__________________________________________________________________________________________________
model_27 (Model)                (None, 20, 1)        481620      input_49[0][0]            

## Dataset Selection

In [207]:
def select_supervised_samples(n_samples, X, labels, n_classes):
    """Select a supervised subset of the dataset, ensures classes are balanced."""
    X_list, y_list = list(), list()
    n_per_class = int(n_samples / n_classes)
    for i in range(n_classes):
        X_with_class = X[labels == i]
        ix = randint(0, len(X_with_class), n_per_class)
        [X_list.append(X_with_class[j]) for j in ix]
        [y_list.append(i) for j in ix]
    return asarray(X_list), asarray(y_list)

def generate_real_samples(n_samples, X, labels):
    """Randomly select real samples."""
    ix = randint(0, X.shape[0], n_samples)
    X, labels = X[ix], labels[ix]
    y = ones((n_samples, 1))
    return [X, labels], y

def generate_latent_points(n_samples, latent_dim):
    """Generate points in latent space as input for the generator."""
    return np.random.normal(size=(n_samples, latent_dim, 1))
 
def generate_fake_samples(n_samples, latent_dim, generator):
    """Generate n fake examples with class labels."""
    z = generate_latent_points(n_samples, latent_dim)
    x_ = generator(z)
    y = zeros((n_samples, 1))
    return x_, y

## Training

In [208]:
fake = np.ones((batch_size, 1))
valid = -np.ones((batch_size, 1))
delta = np.ones((batch_size, 1))

indices = np.arange(X_train.shape[0])
for epoch in range(1, epochs + 1):
    np.random.shuffle(indices)
    X_ = X_train[indices]
    y_ = y_train[indices]

    epoch_g_loss = []
    epoch_cx_loss = []
    epoch_cz_loss = []

    minibatches_size = batch_size * iterations_critic
    num_minibatches = int(X_.shape[0] // minibatches_size)

    for i in range(num_minibatches):
        minibatch = X_[i * minibatches_size: (i + 1) * minibatches_size]
        y_minibatch = y_[i * minibatches_size: (i + 1) * minibatches_size]

        for j in range(iterations_critic):
            x = minibatch[j * batch_size: (j + 1) * batch_size]
            y = y_minibatch[j * batch_size: (j + 1) * batch_size]
            z = np.random.normal(size=(batch_size, latent_dim, 1))
            epoch_cx_loss.append(
                critic_x_model.train_on_batch([y, z], [valid, fake, delta]))
            epoch_cz_loss.append(
                critic_z_model.train_on_batch([x, z], [valid, fake, delta]))

        epoch_g_loss.append(
            encoder_generator_model.train_on_batch([x, z], [valid, valid, y]))

    cx_loss = np.mean(np.array(epoch_cx_loss), axis=0)
    cz_loss = np.mean(np.array(epoch_cz_loss), axis=0)
    g_loss = np.mean(np.array(epoch_g_loss), axis=0)
    print('Epoch: {}/{}, [Dx loss: {}] [Dz loss: {}] [G loss: {}]'.format(
        epoch, epochs, cx_loss, cz_loss, g_loss))

In [213]:
def train(generator, critic_x_unsup, critic_x_sup, gan, X, labels, latent_dim, epochs, batch_size):
    
    X_sup, labels_sup = select_supervised_samples(batch_size, X, labels, n_classes)
    num_batches = X.shape[0] // batch_size
    half_batch = batch_size // 2
    
    for e in range(epochs):
       
        critic_x_losses = []
        generator_losses = []
        
        for n in range(num_batches):
            # Train critic x supervised model
            [X_sup_real, y_sup_real], _ = generate_real_samples(half_batch, X_sup, labels_sup)
            critix_x_sup_loss = list(critic_x_sup.train_on_batch(X_sup_real, y_sup_real))

            # Train critic x unsupervised model with real examples
            [X_unsup_real, _], ones_batch = generate_real_samples(half_batch, X, labels)
            critix_x_unsup_real_loss = critic_x_unsup.train_on_batch(X_unsup_real, ones_batch)

            # Train critic x unsupervised model with fake examples
            X_unsup_fake, zeros_batch = generate_fake_samples(half_batch, latent_dim, generator)
            critix_x_unsup_fake_loss = critic_x_unsup.train_on_batch(X_unsup_fake, zeros_batch)

            critic_x_losses.append(critix_x_sup_loss + [critix_x_unsup_real_loss, critix_x_unsup_fake_loss])


            # Train generator
            X_gan, labels_gan = generate_latent_points(batch_size, latent_dim), ones((batch_size, 1))
            generator_loss = gan.train_on_batch(X_gan, labels_gan)
            
            generator_losses.append([generator_loss])
        
        
        critic_x_loss = np.array(critic_x_losses).mean(axis=0)
        generator_loss = np.array(generator_losses).mean(axis=0)
        print(f'epoch: {e}, critic_x: {critic_x_loss}, generator: {generator_loss}')


In [214]:
train(
    generator, 
    critic_x_unsup, 
    critic_x_sup, 
    gan, 
    X_train, 
    labels_train, 
    latent_dim, 
    epochs, 
    batch_size
)

epoch: 0, critic_x: [0.7081537  0.61621094 0.57729495 0.7271075 ], generator: [1.2430649]
epoch: 1, critic_x: [0.62288827 0.65527344 0.6921256  0.46636146], generator: [1.709136]
epoch: 2, critic_x: [0.50837153 0.7236328  0.43268517 0.27707696], generator: [3.4420176]
epoch: 3, critic_x: [0.3701978  0.7988281  0.09752154 0.07441215], generator: [4.580372]


KeyboardInterrupt: 

## Evaluation of unsupervised model

In [ ]:
z_ = encoder.predict(X_train)
y_hat = generator.predict(z_)
critic = critic_x_unsup.predict(y)

# Phase 2: Query Strategies

In [ ]:
# What did the unsupervised GAN say was an anomaly?
z_ = encoder.predict(X_train)
y_hat = generator.predict(z_)
critic = critic_x_unsup.predict(y)

In [ ]:
# What did the supervised GAN get wrong?

# False Negatives
label_hat = critic_x_unsup.predict(X_train[labels==1])

# False Positives
label_hat = critic_x_unsup.predict(X_train[labels==1])

In [ ]:
# How should we explore datasets?

test = np.randint(X_train)

# Phase 3: Annotation Strategy

In [ ]:
# How do we combine labels from different people?